You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current state of Artificial Intelligence
- Mention the rapid advancements and potential impact on various industries

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning algorithms
- Growth of AI in healthcare, finance, and automotive industries
- Integration of AI in everyday products like smart speakers and virtual assistants

2. Key Players in the AI Indust

I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a transformative force in today's technology landscape, with rapid advancements paving the way for unprecedented innovations across various industries. The current state of AI is characterized by breakthroughs in machine learning algorithms, leading to the integration of AI in everyday products and services. As we delve into the future of AI, it's crucial to explore the latest trends, key players in the industry, and noteworthy news shaping the landscape of artificial intelligence.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms have been a driving force behind the evolution of AI, enabling machines to learn from data and improve their performance over time. This has led to the growth of AI in key sectors such as healthcare, finance, and automotive industries, where AI-powered

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a transformative force in today's technology landscape, with rapid advancements paving the way for unprecedented innovations across various industries. The current state of AI is characterized by breakthroughs in machine learning algorithms, leading to the integration of AI in everyday products and services. As we delve into the future of AI, it's crucial to explore the latest trends, key players in the industry, and noteworthy news shaping the landscape of artificial intelligence.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms have been a driving force behind the evolution of AI, enabling machines to learn from data and improve their performance over time. This has led to the growth of AI in key sectors such as healthcare, finance, and automotive industries, where AI-powered solutions are revolutionizing processes and enhancing efficiency. Moreover, the integration of AI in everyday products like smart speakers and virtual assistants has become increasingly prevalent, showcasing the widespread adoption of AI technologies in our daily lives.

## Key Players in the AI Industry

Leading tech giants such as Google, Microsoft, and Amazon are at the forefront of AI research and development, investing heavily in cutting-edge AI technologies and solutions. Additionally, startups are making significant contributions in niche areas of AI, focusing on specialized applications and innovations. Collaboration between tech giants and academia has also played a crucial role in driving further innovation in the AI industry, with joint efforts aimed at pushing the boundaries of AI capabilities and exploring new frontiers.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in natural language processing and computer vision have propelled AI to new heights, enabling machines to interpret and analyze complex data with unprecedented accuracy. However, ethical considerations in AI development and deployment have emerged as a critical issue, raising questions about the responsible use of AI technologies and the potential impact on society. Moreover, AI applications are increasingly being leveraged to tackle global challenges such as climate change and healthcare disparities, highlighting the transformative potential of AI in addressing pressing issues facing our world.

## Audience Analysis

Our target audience includes tech enthusiasts, business professionals, and students interested in AI, seeking to stay updated on the latest technology trends and understand AI's potential impact on society. By exploring the future of AI technology and its implications, readers can gain valuable insights into the opportunities and challenges presented by the AI revolution. Furthermore, exploring career opportunities in AI can provide a roadmap for individuals looking to pursue a career in this rapidly evolving field.

## Call to Action

We encourage readers to subscribe to our blog for future updates on Artificial Intelligence, ensuring they stay informed about the latest trends and developments in the AI industry. Additionally, we prompt them to explore related articles on AI applications and career opportunities, providing valuable resources and insights to support their journey into the world of artificial intelligence.

In conclusion, the future of Artificial Intelligence is brimming with possibilities, driven by the latest trends, key players, and noteworthy news shaping the industry. By staying informed and engaged with the evolving landscape of AI, we can harness the transformative power of artificial intelligence to drive innovation, address global challenges, and shape a brighter future for generations to come.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "compound AI systems"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on compound AI systems.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Unveiling the Power of Compound AI Systems

Introduction:
- Definition of compound AI systems
- Importance and benefits of using compound AI systems
- Brief overview of key players in the industry

Key Points:
1. Latest Trends in Compound AI Systems:
- Advancements in machine learning algorithms
- Integration of AI with IoT and big data analytics
- Ethical considerations in developing compound AI systems

2. Key Players in Compound AI Systems:
- Google's DeepMind
- IBM Watson
- Microsoft Az

I now can give a great answer

Final Answer:
# Unveiling the Power of Compound AI Systems

## Introduction
In the rapidly evolving landscape of artificial intelligence, compound AI systems have emerged as a powerful tool that combines multiple AI technologies to achieve more complex tasks and deliver superior results. These systems leverage advancements in machine learning algorithms, integration with IoT and big data analytics, and ethical considerations to drive innovation and efficiency in various industries. Key players such as Google's DeepMind, IBM Watson, Microsoft Azure AI, and Amazon Web Services AI are at the forefront of developing and implementing compound AI solutions that are shaping the future of AI technology.

## Latest Trends in Compound AI Systems
One of the most notable trends in compound AI systems is the continuous advancements in machine learning algorithms. These algorithms are becoming more sophisticated and efficient, enabling AI systems to learn and adapt in 

In [14]:
Markdown(result)

# Unveiling the Power of Compound AI Systems

## Introduction
In the rapidly evolving landscape of artificial intelligence, compound AI systems have emerged as a powerful tool that combines multiple AI technologies to achieve more complex tasks and deliver superior results. These systems leverage advancements in machine learning algorithms, integration with IoT and big data analytics, and ethical considerations to drive innovation and efficiency in various industries. Key players such as Google's DeepMind, IBM Watson, Microsoft Azure AI, and Amazon Web Services AI are at the forefront of developing and implementing compound AI solutions that are shaping the future of AI technology.

## Latest Trends in Compound AI Systems
One of the most notable trends in compound AI systems is the continuous advancements in machine learning algorithms. These algorithms are becoming more sophisticated and efficient, enabling AI systems to learn and adapt in real-time to changing environments and data inputs. The integration of AI with IoT and big data analytics is also driving innovation in compound AI systems, allowing for the seamless integration of AI technologies with connected devices and vast amounts of data for more accurate and insightful decision-making. Additionally, ethical considerations in developing compound AI systems have become a key focus, ensuring that AI technologies are developed and deployed responsibly and ethically.

## Key Players in Compound AI Systems
Google's DeepMind, known for its groundbreaking work in AI research and development, is a key player in the field of compound AI systems. IBM Watson, with its cognitive computing capabilities, is revolutionizing industries with its AI-powered solutions. Microsoft Azure AI and Amazon Web Services AI are also leading the way in developing innovative compound AI systems that are driving digital transformation across various sectors.

## Noteworthy News in Compound AI Systems
Recent breakthroughs in natural language processing have opened up new possibilities for compound AI systems, enabling more seamless interactions between humans and machines. The applications of compound AI in healthcare and finance sectors are also gaining traction, with AI technologies being used to improve patient care, streamline financial processes, and enhance decision-making. Regulatory updates on AI technologies are also shaping the landscape of compound AI systems, ensuring that AI technologies are developed and deployed in a responsible and transparent manner.

As professionals in the tech industry, data scientists, and AI enthusiasts, it is essential to stay updated on the latest AI trends and developments, seek insights on how to leverage AI for business growth, and address concerns about the ethical implications of AI technologies. By exploring the potential of compound AI systems and staying informed about the latest advancements in the field, we can harness the power of AI technology to drive innovation and success in our respective industries.

Remember to subscribe for more updates on AI trends and developments, and join us in the journey towards unlocking the full potential of compound AI systems for your business.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).